In [2]:
# train_yolov8_sem.py
from ultralytics import YOLO
import yaml
import os

# -------------------------------
# Paths
# -------------------------------
DATA_YAML = "sem.yaml"
PROJECT = "runs_defect"
RUN_NAME = "yolov8n_sem_256_v2"

# -------------------------------
# Sanity check (optional but recommended)
# -------------------------------
def check_labels(label_dir):
    bad = False
    for root, _, files in os.walk(label_dir):
        for f in files:
            if not f.endswith(".txt"):
                continue
            p = os.path.join(root, f)
            with open(p) as fh:
                for ln in fh:
                    if not ln.strip():
                        continue
                    cid = int(ln.split()[0])
                    if cid < 0 or cid > 6:
                        print("BAD CLASS ID:", cid, "in", p)
                        bad = True
    if not bad:
        print("Label sanity OK")

check_labels("dataset/labels")

# -------------------------------
# Model
# -------------------------------
# Use pretrained backbone (recommended)
model = YOLO("yolov8n.pt")

# -------------------------------
# Train
# -------------------------------
model.train(
    data=DATA_YAML,

    # image & batch
    imgsz=256,
    batch=16,
    epochs=100,

    # device
    device=0,
    workers=4,

    # optimizer (SEM-friendly)
    optimizer="AdamW",
    lr0=3e-4,
    lrf=0.01,
    weight_decay=5e-4,

    # SEM-safe augmentations
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    mosaic=0.15,
    mixup=0.0,
    copy_paste=0.0,
    degrees=0.0,
    translate=0.1,
    scale=0.4,
    shear=0.0,
    perspective=0.0,

    # loss balance (thin / low-contrast defects)
    box=7.5,
    cls=0.6,
    dfl=1.5,

    # stability
    warmup_epochs=3,
    patience=25,
    deterministic=True,

    # logging
    project=PROJECT,
    name=RUN_NAME,
    save=True,
    verbose=True,
)


Label sanity OK
Ultralytics 8.4.12  Python-3.10.0 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5050 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.6, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=sem.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.15, multi_scale=0.0, name=yolov8n_sem_256_v2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020B047FD810>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [1]:
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Compute Capability: {torch.cuda.get_device_capability(0)}")


PyTorch Version: 2.10.0+cu128
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 5050 Laptop GPU
Compute Capability: (12, 0)
